![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.add_equity("SPY")
# Locally Lean installs free sample data, to download more data please visit https://www.quantconnect.com/docs/v2/lean-cli/datasets/downloading-data 
qb.set_start_date(2013, 10, 11)
history = qb.history(qb.securities.keys(), 360, Resolution.DAILY)

# Indicator Analysis
bbdf = qb.indicator(BollingerBands(30, 2), spy.symbol, 360, Resolution.DAILY)
bbdf.drop('standarddeviation', axis=1).plot()

TRAINING VISUALS

project/
└── UMI/
    ├── main.py                  # (patched)
    ├── modules.py               # contains StockLevelFactorLearning, …
    ├── quick_view.ipynb         # Jupyter notebook
    ├── data/                    # raw OHLCV csv files
    │   ├── AAPL.csv
    │   └── …
    └── models/                  # ← will be auto-created on first run
        └── UMI/
            └── 1d/2025-06-22T14-30-24/
                ├── bt_pred_close.csv
                ├── bt_truth_close.csv
                └── …

In [ ]:
from pathlib import Path
import json, torch, pandas as pd
import matplotlib.pyplot as plt
import pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# find the newest run directory
runs = sorted((Path(__file__).parent / "models" / "UMI" / "1d").iterdir(),
              key=lambda p: p.stat().st_mtime)
run_dir = runs[-1]

MODEL_PATH = "1d/2025-06-21T12-00-00"

In [ ]:
# Model-properties overview  ────────────────────────────────────────────

# ‣ point to the same run you loaded the loss curves from
model_dir = Path(f"models/UMI/1d/{MODEL_PATH}")
state_file   = model_dir / "best.pt"
hparams_file = model_dir / "hparams.json"
param_inv_csv = model_dir / "param_inventory.csv"       
hp_trials_csv = model_dir / "hp_trials.csv"             

# ── A. hyper-parameter dictionary ─────────────────────────────────────────
with hparams_file.open() as fp:
    hparams = json.load(fp)

hp_df = pd.DataFrame(hparams.items(), columns=["param", "value"])
display(hp_df.style.set_caption("Saved hyper-parameters"))

# ── B. parameter inventory  (counts + memory) ────────────────────────────
state = torch.load(state_file, map_location="cpu")

records, total_params, total_bytes = [], 0, 0
for top_key, block in state.items():                 # e.g. 'stock_factor', …
    if isinstance(block, dict):                      # sub-module dict
        for name, tensor in block.items():
            full_name = f"{top_key}.{name}"
            n = tensor.numel()
            b = n * tensor.element_size()
            records.append((full_name, list(tensor.shape), n, b))
            total_params += n
            total_bytes  += b
    else:                                            # flat tensor
        n = block.numel()
        b = n * block.element_size()
        records.append((top_key, list(block.shape), n, b))
        total_params += n
        total_bytes  += b

param_df = pd.read_csv(param_inv_csv)

# pretty display – show the 20 largest tensors first
display(param_df.head(20).style.set_caption("Largest tensors (by #params)"))

print(f"Total learnable parameters : {total_params:,}")
print(f"≈ Model size (fp32)        : {total_bytes/1024/1024:.1f} MB")

# ── C. optional: visual snapshot of the heaviest tensors ──────────────────
top = param_df.head(10)           # top-10 by param count
plt.figure(figsize=(12, 4))
plt.bar(top["tensor"], top["numel"])
plt.ylabel("# parameters")
plt.title("Top-10 tensors by parameter count")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


In [ ]:

pred  = pd.read_csv(run_dir / "bt_pred_close.csv",  index_col=0, parse_dates=True)
truth = pd.read_csv(run_dir / "bt_truth_close.csv", index_col=0, parse_dates=True)

ticker = "AAPL"

fig, ax = plt.subplots()
ax.plot(truth.index, truth[ticker], label="real close")
ax.plot(pred.index,  pred[ticker],  label="predicted close")
ax.set_title(f"{ticker} – 1-bar-ahead close forecast")
ax.legend()
plt.show()
